In [0]:
# Importing neccessary packages
import numpy as np
import tensorflow as tf
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
os.listdir('.')

['.config',
 'evaluation',
 'food11.zip',
 'validation',
 'kaggle.json',
 'drive',
 'training',
 'sample_data']

### Data preparation

The dataset is seperated into 3 folders: training, validation and evaluation.
The images are named by convention as: {class_id}-{image_id}.jpg. Create subfolders inside training and validation for ImageDataGenerator to recognize classes.

In [0]:
NUM_CLASSES = 11

In [0]:
classes_list = [
    'Bread',
    'Dairyproduct', 
    'Dessert', 
    'Egg', 
    'Friedfood', 
    'Meat', 
    'NoodlesPasta', 
    'Rice', 
    'Seafood', 
    'Soup', 
    'VegetableFruit'
]

In [0]:
classes_list

['Bread',
 'Dairyproduct',
 'Dessert',
 'Egg',
 'Friedfood',
 'Meat',
 'NoodlesPasta',
 'Rice',
 'Seafood',
 'Soup',
 'VegetableFruit']

In [0]:
classes_list[0]

'Bread'

In [0]:
# Data augmentation for regularization and adding some extra training samples

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True, 
    zoom_range=0.2,
    shear_range=0.2
)

In [0]:
TARGET_DIM = 300
BATCH_SIZE = 32

In [0]:
train_generator = datagen.flow_from_directory(
    directory='./training/', 
    target_size=(TARGET_DIM, TARGET_DIM),
    batch_size=BATCH_SIZE,
)

validation_generator = datagen.flow_from_directory(
    directory='./validation/',
    target_size=(TARGET_DIM, TARGET_DIM),
    batch_size=BATCH_SIZE
)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [0]:
train_generator.class_indices

{'Bread': 0,
 'Dairyproduct': 1,
 'Dessert': 2,
 'Egg': 3,
 'Friedfood': 4,
 'Meat': 5,
 'NoodlesPasta': 6,
 'Rice': 7,
 'Seafood': 8,
 'Soup': 9,
 'VegetableFruit': 10}

In [0]:
validation_generator.class_indices

{'Bread': 0,
 'Dairyproduct': 1,
 'Dessert': 2,
 'Egg': 3,
 'Friedfood': 4,
 'Meat': 5,
 'NoodlesPasta': 6,
 'Rice': 7,
 'Seafood': 8,
 'Soup': 9,
 'VegetableFruit': 10}

In [0]:
# Lets try the Resnet152 architecture

base_model = tf.keras.applications.resnet.ResNet152(
    include_top=False, 
    weights='imagenet', 
    input_shape=(TARGET_DIM, TARGET_DIM, 3)
)

In [0]:
base_model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [0]:
print('Layers in Resnet152: ' + str(len(base_model.layers)))

Layers in Resnet152: 515


In [0]:
preds = base_model.output
preds = tf.keras.layers.GlobalAveragePooling2D()(preds)
preds = tf.keras.layers.Dense(512, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(256, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(128, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.Dense(11, activation=tf.nn.softmax)(preds)

In [0]:
model = tf.keras.models.Model(base_model.input, preds)

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
model.layers[-7].name

'global_average_pooling2d_1'

In [0]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [0]:
# Create directory in drive for storing model
!mkdir drive/My\ Drive/food_11_weights

In [0]:
# Lets define checkpoint for model saving
filepath="./drive/My Drive/food_11_weights/resnet152-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [0]:
# Freeze mobilenet layers and train only newly added layers

for layers in model.layers[:-7]:
    layers.trainable = False

for layers in model.layers[-7:]:
    layers.trainable = True

In [0]:
#Let's train the model 20 epochs as it will take a lot of time

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint],
    epochs=20
)

Epoch 1/20
308/308 [==============================] - ETA: 0s - loss: 2.3055 - acc: 0.1877
Epoch 00001: val_acc improved from -inf to 0.09521, saving model to ./drive/My Drive/food_11_weights/resnet152-01-0.10.hdf5
308/308 [==============================] - 590s 2s/step - loss: 2.3055 - acc: 0.1877 - val_loss: 2.3745 - val_acc: 0.0952
Epoch 2/20
308/308 [==============================] - ETA: 0s - loss: 2.1307 - acc: 0.2510
Epoch 00002: val_acc improved from 0.09521 to 0.14603, saving model to ./drive/My Drive/food_11_weights/resnet152-02-0.15.hdf5
308/308 [==============================] - 581s 2s/step - loss: 2.1307 - acc: 0.2510 - val_loss: 2.6418 - val_acc: 0.1460
Epoch 3/20
308/308 [==============================] - ETA: 0s - loss: 2.0563 - acc: 0.2787
Epoch 00003: val_acc improved from 0.14603 to 0.21817, saving model to ./drive/My Drive/food_11_weights/resnet152-03-0.22.hdf5
308/308 [==============================] - 582s 2s/step - loss: 2.0563 - acc: 0.2787 - val_loss: 10.1354 